In [1]:
import numpy as np
import cv2

In [2]:
# read depth image
depth_scale = 0.00012498664727900177
depth_img = cv2.imread('depth.png') # cv2.imread 得到 BGR 图像
dpt = depth_img[:, :, 2] + depth_img[:, :, 1] * 256
dpt = dpt * depth_scale # (480, 640)

# read seg image
seg = cv2.imread('seg.png')[...,0]  # 255: fore ground, 0: background
# (480, 640)

# read intrinsics and extrinsics
K = np.load('intrinsic.npy')
print(K)

[[415.69219382   0.         320.        ]
 [  0.         415.69219382 240.        ]
 [  0.           0.           1.        ]]


In [3]:
# task1: convert depth image to point cloud
# 利用 intrinsics & extrinsics 从二维图像重建3d坐标，再根据 depth 进行调整
def depth2pc(depth, seg, K):
    H, W = dpt.shape
    y = np.tile(np.arange(H).reshape(-1, 1), W).reshape(H, W, 1)
    x = np.tile(np.arange(W), (H, 1)).reshape(H, W, 1)
    ones = np.ones((H, W, 1))
    img = np.concatenate((x, y, ones), axis=-1)
    img = img[np.where(seg>0)]
    depth = depth[np.where(seg>0)].reshape(-1, 1)
    pc = np.linalg.inv(K).dot(img.T).T
    pc = pc*depth
    return pc

partial_pc = depth2pc(dpt, seg, K)

# For debug and submission
np.savetxt('../results/pc_from_depth.txt', partial_pc)

In [4]:
# task2: compute one-way chamfer distance to the complete shape
# Note that 'full_pc.txt' is the point cloud sampled from the mesh before rendering to depth image. 
# Since the default direction of the camera in PyRender is (0,0,-1), you may find the results of your back projection is not aligned with 'full_pc.txt'.
# To avoid confusion, we provide 'aligned_full_pc.txt', which should be aligned with your resulted point cloud.
full_pc = np.loadtxt('aligned_full_pc.txt')

def random_sample(pc, num):
    permu = np.random.permutation(pc.shape[0])
    return pc[permu][:num]

partial_pc_sampled = random_sample(partial_pc, 2048) # (2048,3)
full_pc_sampled = random_sample(full_pc, 2048)


# 先每个点之间两两计算距离，组成2048*2048矩阵
P = np.tile(partial_pc_sampled.reshape(2048, 1, 3), (1, 2048, 1))
F = np.tile(full_pc_sampled.reshape(1, 2048, 3), (2048, 1, 1))
dist = np.apply_along_axis(np.linalg.norm, 2, P-F)
one_way_CD = np.sum(np.min(dist, axis=1)) / 2048

print('one way chamfer distance: ', one_way_CD)

one way chamfer distance:  0.009852734259849272


In [5]:
# For submission
np.savetxt('../results/one_way_CD.txt', one_way_CD.reshape(1))